In [1]:
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

import geopandas as gpd
import pandas as pd
import glob
import zipfile
import rasterio
import os
import xarray as xr
import json

Populating the interactive namespace from numpy and matplotlib


In [2]:
print(os.getcwd())
from pathlib import Path
folder = Path(r'../Data') # assumes working directory is onhm-fetcher-parser
print(folder)
shapefiles = folder.glob("*_0[1].shp")
#shapefiles = folder.glob("*.shp")
gdf = pd.concat([
    gpd.read_file(shp)
    for shp in shapefiles
]).pipe(gpd.GeoDataFrame)
gdf.reset_index(drop=True, inplace=True)
# gdf.plot()
print(gdf)

B:\GitRepos\onhm-fetcher-parser\notebooks
..\Data
       POI_ID  hru_id_nat  hru_id_reg region  \
0     7733855           1           1     01   
1     7733919           2           2     01   
2     7732571           3           3     01   
3     7732387           4           4     01   
4     7733327           5           5     01   
...       ...         ...         ...    ...   
2457   719280        2458        2458     01   
2458   805345        2459        2459     01   
2459   806793        2460        2460     01   
2460   807073        2461        2461     01   
2461   725382        2462        2462     01   

                                               geometry  
0     POLYGON ((-73.37148 41.11233, -73.37151 41.112...  
1     POLYGON ((-73.38532 41.13467, -73.38566 41.134...  
2     POLYGON ((-73.41947 41.16068, -73.41955 41.160...  
3     POLYGON ((-73.41097 41.15833, -73.41105 41.158...  
4     MULTIPOLYGON (((-73.28636 41.12781, -73.28645 ...  
...                      

In [73]:
wght_UofI = pd.read_csv('weights.csv')
print(wght_UofI.head())
wghts_id = wght_UofI.columns[1]
print(wghts_id)

   grid_ids  hru_id_nat         w
0    278433           1  0.000120
1    277046           1  0.031155
2    277047           1  0.901587
3    277048           1  0.046570
4    275661           1  0.020568
hru_id_nat


In [83]:
import numpy.ma as ma
import numpy as np

index = gdf.hru_id_nat.values
gid = wght_UofI.grid_ids.values
hid = wght_UofI.hru_id_nat.values
w = wght_UofI.w.values

def getweights(hruind, mgid, mhid, mw):
    tmask = np.where(mhid == hruind)
    return tmask, mgid[tmask], mw[tmask]
    
tmask, tgid,tw = getweights(index[0], gid, hid, w)
print(index[0], tgid, thid, tw)
print(tw.sum())
tmask

1 [278433 277046 277047 277048 275661] [1 1 1 1 1] [1.20332377e-04 3.11551394e-02 9.01586747e-01 4.65699876e-02
 2.05676159e-02]
0.9999998227064826


(array([0, 1, 2, 3, 4], dtype=int64),)

In [81]:
for tind, val in np.ndenumerate(index):
    if tind[0] < 11:
        tmask, tgid, tw = getweights(val, gid, hid, w)
        print(tind[0], tmask, tgid, tw, tw.sum())

0 [278433 277046 277047 277048 275661] [1.20332377e-04 3.11551394e-02 9.01586747e-01 4.65699876e-02
 2.05676159e-02] 0.9999998227064826
1 [277046 277047 275660 275661] [0.13172276 0.17021142 0.27964766 0.41841768] 0.9999995228417933
2 [275660] [1.] 1.0
3 [275660 275661] [0.82125035 0.17874879] 0.9999991394065904
4 [277049 277050 275663] [2.88188403e-01 7.11235395e-01 5.75810834e-04] 0.9999996090536466
5 [275663] [1.] 1.0
6 [275660 274274] [0.93762659 0.06237332] 0.9999999102804411
7 [278432 278433 277046 277047 277045 275658 275659 274273 274272] [0.02521717 0.04948689 0.09536192 0.22955413 0.00280794 0.25261284
 0.32046514 0.02088624 0.00360742] 0.999999693481975
8 [277049 277050 275664 275663] [0.15569906 0.40014603 0.00602082 0.43813344] 0.9999993467119019
9 [275664 275663] [0.89677002 0.10322969] 0.9999997151658577
10 [277051 277050 275665 275664] [0.21947723 0.57669696 0.00457921 0.19924623] 0.999999625768125
